<h1>Estimating Monetary Policy Reaction Function (M-H Algorithm)<h1\>

 <h5> In a typical monetary policy reaction function, central banks set interest rates based on variables such as inflation, output gap, or unemployment. The reaction function can be modeled as:
$$
\text{FFR}_t = \rho \cdot \text{FFR}_{t-1} + (1 - \rho) \left( c + \beta_1 \cdot \text{Inflation}_{t-1} + \beta_2 \cdot \text{UN}_{t-1} \right) + \varepsilon_t
$$

where:

- $ \text{FFR}_t $ is the policy interest rate (e.g., the Federal Funds Rate),
- $ \rho $ is the smoothing parameter, showing the degree of inertia in rate-setting,
- $ c $ is a constant term,
- $ \beta_1 $ is the response coefficient to inflation,
- $ \beta_2 $ is the response coefficient to unemployment,
- $ \varepsilon_t $ is a random error term with variance $ \sigma^2 $.
 \sigma^2 $.
$ \sigma^2 $.log-likelihood. <h5\>\(\sigma^2\).


<h2>Load library<h2\>

In [1]:
import numpy as np
from scipy.stats import beta, norm, invgamma
import matplotlib.pyplot as plt
import pandas as pd




<h2>Load data<h2\>

In [7]:
# Load data
data = pd.read_excel('./Data.xlsx').values
T = data.shape[0]

# Extract variables
FFR = data[:, 0]  # Federal funds rate
Inflation = data[:, 1]  # Inflation rate
UN = data[:, 2]  # Unemployment rate


<h2>Set parameters<h2\>

$$
\rho \sim \text{Beta}(10, 10), \quad c \sim \text{Normal}(5, 9), \quad \beta_1 \sim \text{Normal}(1, 0.25)
,\quad \beta_2 \sim \text{Normal}(-1, 0.25), \quad \sigma^2 \sim \text{InverseGamma}(4, 12)
$$


In [ ]:

# Specification dictionary
Spec = {
    'Data': data,
    'MH': 2,
    'nu': 15,
    'alpha0': 10,             # beta distribution
    'beta0': 10,              # beta distribution
    'Normal_mu': np.array([5, 1, -1]),                  # normal distribution
    'Normal_V': np.array([9, 0.25, 0.25]),              # normal distribution
    'a0': 8,                  # inverse gamma distribution
    'd0': 24,                 # inverse gamma distribution
    'ind_beta': 1,              # index pointing to the Beta distribution 
    'ind_Normal': [2, 3, 4],    # index pointing to the normal distribution
    'ind_IG': 5                 # index pointing to the inverse gamma distribution
}


<h2>Define log-likelihood function<h2\>

The log-likelihood function compares the actual observed policy rate with the rate implied by the reaction function for each time period:

1. Calculate Expected Policy Rate:

   Based on the reaction function and the current parameter values, compute the expected interest rate for each period
   , given the lagged inflation and unemployment values.

2. Calculate Residuals:

   The residual is the difference between the observed and the expected rate 

3. Construct Log-Likelihood:

   Assuming Gaussian errors, the log-likelihood of observing each rate, given the parameters, is:
   
   $$
   \ln L_t = -\frac{1}{2} \ln(2 \pi \sigma^2) - \frac{(\text{FFR}_t - \text{Ybar}_t)^2}{2 \sigma^2}
   $$
   
   Summing this over all observations provides the total log-likelihood:
   
   $$
   \text{lnlik} = \sum_{t} \ln L_t
   $$

This total log-likelihood reflects the fit of the reaction function to the observed data.

In [ ]:
# The model assumes that the errors (or residuals) between the observed interest rate (FFR_t) and the predicted interest rate (Ybar_t)
# follow a normal (Gaussian) distribution. Taking the natural log of the likelihood (to make it easier to work with) 
# results in the log-likelihood for each observation. Lastly the total log-likelihood is obtained by summing the individual 
# log-likelihoods across all observations 

def lnlik(theta, Spec):
    Data = Spec['Data']
    FFR, Inflation, UN = Data[:, 0], Data[:, 1], Data[:, 2]

    rho, c, beta1, beta2, sig2 = theta
    T = len(FFR)

    # Define Yt and Ybar based on model
    Yt = FFR[1:T]
    Ybar = (1 - rho) * c + (1 - rho) * beta1 * Inflation[:T-1] + (1 - rho) * beta2 * UN[:T-1] + rho * FFR[:T-1]
    
    # Compute log-likelihood
    sigma = sig2 * np.ones(T - 1)
    lnLm = -0.5 * np.log(2 * np.pi * sigma) - 0.5 * ((Yt - Ybar) ** 2 / sigma)
    return np.sum(lnLm)

<h2>Define Log-prior function<h2\>

In [ ]:
# This function calculates the log of the prior probability density for each parameter based on their specified distributions:

def lnprior(theta, Spec):
    # Unpack hyperparameters
    alpha0, beta0 = Spec['alpha0'], Spec['beta0']
    Normal_mu, Normal_V = Spec['Normal_mu'], Spec['Normal_V']
    a0, d0 = Spec['a0'], Spec['d0']

    # Unpack parameter indices
    ind_beta = Spec['ind_beta']
    ind_Normal = Spec['ind_Normal']
    ind_IG = Spec['ind_IG']

    # Priors
    lnPrior = beta.logpdf(theta[ind_beta], alpha0, beta0)  # Beta prior for rho
    lnPrior += np.sum(norm.logpdf(theta[ind_Normal], Normal_mu, np.sqrt(Normal_V)))  # Normal priors for (c, beta1, beta2)
    lnPrior += invgamma.logpdf(theta[ind_IG], a0 / 2, scale=d0 / 2)  # Inverse-Gamma prior for sig2
    
    return lnPrior

# log-posterior

def lnpost(theta, Spec):
    lnL = lnlik(theta, Spec)
    lnPrior = lnprior(theta, Spec)
    return lnL + lnPrior

<h2>Define parameter constraints
    <h2\>

In [ ]:
def paramconst(theta, Spec):
    # Check that rho is between 0 and 1 and sig2 > 0
    valid = True
    valid &= (0 < theta[0] < 1)  # rho in (0,1)
    valid &= (theta[4] > 0)  # sig2 > 0
    return valid


<h2>Define Metropolis-Hastings Sampler<h2\>

In [10]:
# The function MCMC performs Markov Chain Monte Carlo (MCMC) sampling using the Metropolis-Hastings algorithm
# to approximate the posterior distribution of a set of parameters based on a given log-posterior function.


def MCMC(lnpost, paramconst, n0, n1, theta_hat, V, freq, Spec):
    samples = []
    theta = theta_hat
    accepted = 0

    for i in range(n0 + n1):
        # Propose new sample from multivariate normal around current theta
        proposal = np.random.multivariate_normal(theta, V)

        # Check parameter constraints
        if not paramconst(proposal, Spec):
            continue

        # Calculate acceptance ratio
        log_posterior_current = lnpost(theta, Spec)
        log_posterior_proposal = lnpost(proposal, Spec)
        acceptance_ratio = np.exp(log_posterior_proposal - log_posterior_current)

        # Accept or reject
        if np.random.rand() < acceptance_ratio:
            theta = proposal
            accepted += 1

        # Store sample if beyond burn-in period and at the right frequency
        if i >= n0 and (i - n0) % freq == 0:
            samples.append(theta)

    samples = np.array(samples)
    acceptance_rate = accepted / (n0 + n1)
    return samples, acceptance_rate

<h2>Run MCMC Sampler<h2\>

In [ ]:
# Initial values and proposal covariance matrix for MCMC
theta0 = np.array([0.5, *Spec['Normal_mu'], 0.5 * Spec['a0'] / (0.5 * Spec['d0'] - 1)])
V = np.diag([0.1] * len(theta0))  # Simplified covariance matrix, may require tuning

# MCMC parameters
n0 = 2000  # Burn-in period
n1 = 20000  # Number of MCMC samples
freq = 500  # Frequency of storing samples

# Run MCMC
samples, acceptance_rate = MCMC(lnpost, paramconst, n0, n1, theta0, V, freq, Spec)
print("Acceptance rate:", acceptance_rate)

<h2>Plot<h2\>

In [ ]:
# Plotting prior and posterior for the first parameter as an example
prior = 1 + np.random.randn(len(samples))
plt.figure(figsize=(8, 5))
plt.hist(prior, bins=30, density=True, alpha=0.5, label='Prior', linestyle='--')
plt.hist(samples[:, 2], bins=30, density=True, alpha=0.7, label='Posterior')
plt.xlabel(r'$\beta_1$')
plt.ylabel('Density')
plt.legend(loc='upper right')
plt.show()